<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/Mediclal_Paper_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Required Libraries

In [3]:
! pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 17.8 MB/s eta 0:00:00


In [4]:
! pip install --quiet sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.7 MB/s eta 0:00:00


In [5]:
!pip install --quiet bert-extractive-summarizer

In [6]:
! pip --quiet install sumy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 75.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [1]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [7]:
!pip install --quiet PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 33.7 MB/s eta 0:00:00


## Import Necessary Packages

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
#
from transformers import pipeline
#
from summarizer import Summarizer
from summarizer import TransformerSummarizer
#
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
#
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
nltk.download('brown')
nltk.download('punkt')
#
import fitz
from operator import itemgetter
#
import textwrap
#
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


cuda


## PDF Processing

####PDF - Extracting Text

In [9]:
# Load the PDF document using PyMuPDF
doc = fitz.open("/content/Published.SAFPJ.11.05.2017.pdf")


In [10]:
type(doc)

fitz.fitz.Document

#### Extract Headers and paragraphs from PDF

In [11]:
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

In [12]:
def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag

In [13]:
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)

    return header_para

####Function to extract text from PDF for respective headers

In [14]:
#Function to extract text from PDF for respective headers
def extractTextPdf(pdf_path):
  
    document = pdf_path
    doc = fitz.open(document)
    #
    font_counts, styles = fonts(doc, granularity=False)
    #
    size_tag = font_tags(font_counts, styles)
    #
    elements = headers_para(doc, size_tag)
    print(elements)
    
    return elements



In [15]:
def header_text(edict):
  element_dict = {}
  for text in ele:
    #print(text)
    #prev = text[0:start]
   
    if text.startswith("<h"): 
      start = text.index(">")
      header = text[start+1:].replace("|","")
    
    if text.startswith("<p") and header not in element_dict.keys():
      c = []
      start = text.index(">")
      c.append(text[start+1:].replace("|",""))
      element_dict[header] = c
    elif text.startswith("<p") and header in element_dict.keys():
      start = text.index(">")
      c.append(text[start+1:].replace("|",""))
      element_dict[header] =c
    else:
      continue
  for k,v in element_dict.items():
        p = "".join(v)
        rec = f"{k}:{p}".strip()
        print(rec)
        rec = rec+"\n\n"
        with open('Pdf_extracted_text.txt',"a+",encoding="utf-8") as f:
          f.write(rec)
          f.write("\n")
  return element_dict


## Extract Text from the PDF and return thr header and paragraphs

In [16]:
#
pdf_path = "/content/Published.SAFPJ.11.05.2017.pdf"
ele = extractTextPdf(pdf_path)
#
#Prepare a dictionary with header and corresponding paragrapgh details
#
element_dict = header_text(ele)


['<s8>See discussions, stats, and author profiles for this publication at:  https://www.researchgate.net/publication/316990409|', '', '<h2>The medical management of glaucoma|', '', '<s6>Article in South African Family Practice · May 2017|', '', '<s12>DOI: 10.4102/safp.v59i2.4669|', '', '<s10>CITATIONS|', '<s2>3|', '', '<s10>READS|', '<s2>11,056|', '', '<s6>2 authors:|', '<s6>Some of the authors of this publication are also working on these related projects:|', '<s6>Medical View project|', '<s6>Indigenous Knowledge Systems: Pharmacology and Toxicology View project|', '<s6>Andre Marais|', '<s6>University of Pretoria|', '', '<s8>20', '<s10>PUBLICATIONS', '<s8>34', '<s10>CITATIONS|', '<s10>SEE PROFILE|', '', '<s6>Elzbieta Osuch|', '<s6>Sefako Makgatho Health Sciences University|', '', '<s8>26', '<s10>PUBLICATIONS', '<s8>110', '<s10>CITATIONS|', '<s10>SEE PROFILE|', '', '<s6>All content following this page was uploaded by  Andre Marais  on 17 May 2017.|', '', '<s10>The user has requested en

## Transformers Pipeline - bart-large-cnn

* Bart the max_length is 1024 and for T5 it's 512. 
* "bart-large-cnn" is the default model for the summarization pipeline

In [18]:
summarizer = pipeline("summarization", model='facebook/bart-large-cnn',min_length=75,max_length=500)

In [22]:
from tqdm import tqdm
sum_pipeline = []
for header,sentence in tqdm(element_dict.items()):
  sent = " ".join(element_dict[header])
  #print(sent)
  summary_text = summarizer(sent,truncation=True)[0]['summary_text']
  #print(f"summary text : {summary_text}")
  prompt = header+":"+summary_text
  sum_pipeline.append(prompt)

100%|██████████| 16/16 [05:16<00:00, 19.79s/it]


In [23]:
with open("summary_pipeline_bart_large_cnn.txt","a+",encoding='utf-8') as f:
  for item in sum_pipeline:
    f.write(item)
    f.write("\n")

## facebook/bart-large-xsum

In [24]:
bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-xsum")
bart_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-xsum")

In [25]:
print(bart_tokenizer.model_max_length)

1024


In [27]:
bart_large_xsum_summ = []
for header,sentence in tqdm(element_dict.items()):
  sent = " ".join(element_dict[header])
  print(f"Text :{sent}")
  tokenized_text = bart_tokenizer.encode(sent,return_tensors="pt",max_length=1024,truncation=True)
  max_source_length = max([len(x) for x in tokenized_text])
  print(f"Max source length: {max_source_length}")
  summary = bart_model.generate(tokenized_text,min_length=75,max_length=max_source_length + 100).to(device)
  summary_text = bart_tokenizer.decode(summary[0],skip_special_tokens=True)
  print(f"Summary:{summary_text}")
  #summary_text = ".".join([text.strip().capitalize() for text in summary_text.split(".")])
  prompt = header+":"+summary_text
  bart_large_xsum_summ.append(prompt)

  0%|          | 0/16 [00:00<?, ?it/s]

Text :Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the ey

  6%|▋         | 1/16 [00:21<05:22, 21.52s/it]

Summary:Glaucoma is a disease of the eye that affects the optic nerve and can lead to blindness, according to the American Academy of Ophthalmic Surgeons (AAOS) in the United States and    the European Association for the Study of Ocular Sciences (EAS) in France. . .   and the World Health Organization (WHO) in 2015.
Text :Glaucoma is one of the leading causes of blindness worldwide. The main characteristic features include optic disc cupping, visual field  loss, and in most instances a raised intraocular pressure. Primary open angle glaucoma accounts for nearly two thirds of all cases, while  angle closure glaucoma contributes the majority of the remainder. Screening for glaucoma is not cost-effective, and diagnosis of the  primary open angle type is usually an incidental finding during ophthalmological examination. Various treatment options, including  medical and surgical interventions are accessible in the management of glaucoma. This article contains a brief description on the  pa

 12%|█▎        | 2/16 [00:41<04:50, 20.78s/it]

...

Summary:Timolol is a β blocker used in the treatment of glaucoma and ocular hypertension, as well as in the prevention of cataract and retinitis pigmentosa (RPS) in patients with age-related macular degeneration (AMD) and/or cataracts, and in the management of rheumatoid arthritis (RA).
Text :With the recent attention cannabis has been receiving in the  management of pain and other cancer related conditions, it  is important to remember the favourable effect of its active  ingredient, Δ -tetrahydrocannabinol (THC), in reducing ocular  blood flow. Studies from the 1970s have reported a 30% reduction  in the IOP by inhalation or intravenous administration of THC. The  use is, however, limited owing to the cardiovascular (increased heart  rate and decreased blood pressure) and neurological (tolerance  and addiction) side effects. Topical application does not produce  a significant clinical effect. The use of synthetic cannabinoids may  provide a basis for potential future treatment strate

 94%|█████████▍| 15/16 [05:01<00:17, 17.45s/it]

Summary:With the recent attention cannabis has been receiving in the  management of pain and other cancer related conditions, it is important to remember that cannabis can also be used in the treatment of eye conditions.   .   and the use of synthetic cannabinoids may  provide a basis for potential future treatment strategies for eye conditions, such as glaucoma and retinopathy.
Text :General practitioners and family physicians play an important  role in the diagnosis and appropriate referral of patients with  suspected glaucoma. Fundoscopic examinations are quick and  easy to perform and are an opportunity that present at each  consultation. The early recognition of symptoms may improve  outcomes and delay progression. Taking a good medical history  and applying the principles of rational drug use may identify  potential pharmacological causes for glaucoma. Practitioners  prescribing any of the drugs known to cause acute angle glaucoma  should be vigilant in patients presenting with c

100%|██████████| 16/16 [05:19<00:00, 19.95s/it]

Summary:Glaucoma is one of the leading causes of blindness in adults in the UK, according to a report by the Royal College of Ophthalmologists (RCO)  in the Republic of Ireland, which was published in the British Medical Journal (BMJ) in March 2016, and is estimated to cost the NHS    £1.5bn a year to treat.


In [28]:
bart_large_xsum_summ

['Introduction:Glaucoma is a disease of the eye that affects the optic nerve and can lead to blindness, according to the American Academy of Ophthalmic Surgeons (AAOS) in the United States and    the European Association for the Study of Ocular Sciences (EAS) in France. . .   and the World Health Organization (WHO) in 2015.',
 'Abstract:The main types of glaucoma are primary open angle and angle closure, with the former accounting for more than half of all cases, according to the World Health Organization (WHO) and the International Academy of Ophthalmologists (IAO) in London, England, and the European Association for Ophthalmic Research (EAR) in Paris, France,  and the American Association for the Advancement of Science in Ophthalmology (AAAS).',
 'Corresponding author, email: dramarais@gmail.com / andre.marais@up.ac.za:Glaucoma is a type of eye disease that can be classified into two main types: open angle and closed angle, both of which affect the eyes at the front of the eye, and c

In [29]:
with open("summary_bart_large_xsum.txt","a+",encoding='utf-8') as f:
  for item in bart_large_xsum_summ:
    f.write(item)
    f.write("\n")

## Bert Extractive Summarizer

In [ ]:
from summarizer import Summarizer
model = Summarizer()
bert_summ = []
reduction_rate = 0.1
for header,sentence in tqdm(element_dict.items()):
  sent = " ".join(element_dict[header])
  #print(sent)
  #sentence_tokens = [sent for sent in sent_tokenize(sent)]
  #summary_length = int(len(sentence_tokens)*reduction_rate)
  #print(summary_length)
  summary_text = model(sent,num_sentences=4)
  print(''.join(summary_text))
  summary_text = ''.join(summary_text)
  prompt = header+":"+summary_text
  bert_summ.append(prompt)




Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  6%|▋         | 1/16 [00:00<00:04,  3.65it/s]

Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy. Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated. Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).


 19%|█▉        | 3/16 [00:00<00:02,  4.74it/s]

Glaucoma is one of the leading causes of blindness worldwide. Various treatment options, including  medical and surgical interventions are accessible in the management of glaucoma. This article contains a brief description on the  pathophysiology and focuses on the pharmacological options available in the treatment of glaucoma. Keywords:  Glaucoma, prostaglandin analogs, carbonic anhydrase inhibitors, beta antagonists, alpha 2 adrenergic agonists, cholinergic  agonists
Both types can be further divided into primary versus secondary,  and congenital versus acquired. Glaucoma has a global prevalence  of 3.54% in those aged between 40 and 80 years, and is the  second leading cause of irreversible blindness, only superseded  by cataracts. Open angle glaucoma is the highest in Africa (4.2%),  while closed angle glaucoma is more prevalent in Asia (1.09%).


 25%|██▌       | 4/16 [00:00<00:02,  5.62it/s]

...

Brimonidine is a selective α2 receptor agonist. Apraclonidine  reduces baseline IOP by 30% and is additionally used as adjuvant  therapy in resistant cases of glaucoma. Caution  should be taken in patients with cerebral or coronary insufficiency,  postural hypotension and renal or hepatic failure. Hypersensitivity  reactions, ocular irritation, eyelid edema, foreign body sensation  and dry eyes are common with topical use.
Pilocarpine is the only available cholinergic agent in the  management of acute closed-angle glaucoma registered in South  Africa. Administration results in the increase in aqueous humour  outflow by contraction of the ciliary muscle and constriction of the  pupil. Poorly tolerated side effects, such as ciliary muscle spasm,  myopia and decreased vision limits its usefulness.


 88%|████████▊ | 14/16 [00:04<00:00,  4.48it/s]

Although prostaglandin receptor agonists are the most potent  agents in reducing the IOP, beta blockers are regarded as the first  choice in treating open angle glaucoma. Individual agents differ in  their ability to lower IOP, with betaxolol achieving a 15% decrease,  compared to 20–25% with timolol and levobunolol respectively. Betaxolol is, however, more cardioselective (β1) and therefore has  fewer pulmonary side effects compared to timolol and levobunolol  (which are non-selective antagonists on β1 and β2 receptors). The most common adverse effects  are ocular irritation and dry eyes.
Several combination agents are available in the treatment of  glaucoma. Fixed-dose combination  therapy is more efficacious than its individual components, but still  less efficacious than its respective unfixed combinations. It provides  for simplified treatment regimes and enhances compliance. Combination preparations with brimonidine  have less visual disturbances, taste alterations and ocular all

100%|██████████| 16/16 [00:04<00:00,  3.34it/s]

With the recent attention cannabis has been receiving in the  management of pain and other cancer related conditions, it  is important to remember the favourable effect of its active  ingredient, Δ -tetrahydrocannabinol (THC), in reducing ocular  blood flow. Studies from the 1970s have reported a 30% reduction  in the IOP by inhalation or intravenous administration of THC. The  use is, however, limited owing to the cardiovascular (increased heart  rate and decreased blood pressure) and neurological (tolerance  and addiction) side effects. Topical application does not produce  a significant clinical effect.
General practitioners and family physicians play an important  role in the diagnosis and appropriate referral of patients with  suspected glaucoma. Fundoscopic examinations are quick and  easy to perform and are an opportunity that present at each  consultation. Taking a good medical history  and applying the principles of rational drug use may identify  potential pharmacological cau

In [ ]:
with open("bert_extractive_summarizer.txt","a+",encoding='utf-8') as f:
  for item in bert_summ:
    f.write(item)
    f.write("\n")

## Custom Model- GPT2

In [ ]:
from summarizer import TransformerSummarizer
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")


In [ ]:
gpt2_sum_pipeline = []
for header,sentence in tqdm(element_dict.items()):
  sent = " ".join(element_dict[header])
  #print(sent)
  full = ''.join(GPT2_model(sent, min_length=100))
  prompt = header+":"+full
  gpt2_sum_pipeline.append(prompt)

100%|██████████| 16/16 [00:03<00:00,  4.16it/s]


In [ ]:
with open("bert_extractive_gpt2_summ.txt","a+",encoding='utf-8') as f:
  for item in gpt2_sum_pipeline:
    f.write(item)
    f.write("\n")

## Custom Model- XLNet

In [ ]:
from summarizer import TransformerSummarizer
model = TransformerSummarizer(transformer_type="XLNet",transformer_model_key="xlnet-base-cased")

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
xlnet_sum_pipeline = []
for header,sentence in tqdm(element_dict.items()):
  sent = " ".join(element_dict[header])
  #print(sent)
  full = ''.join(GPT2_model(sent, min_length=100))
  prompt = header+":"+full
  xlnet_sum_pipeline.append(prompt)

100%|██████████| 16/16 [00:03<00:00,  4.19it/s]


In [ ]:
with open("bert_extractive_xlnet_summ.txt","a+",encoding='utf-8') as f:
  for item in xlnet_sum_pipeline:
    f.write(item)
    f.write("\n")

##Transformers -biomedical_text_summarization"



In [ ]:
sbio_tokenizer = AutoTokenizer.from_pretrained("t5-large")
sbio_model = AutoModelForSeq2SeqLM.from_pretrained("sumedh/biomedical_text_summarization")

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
sbio_tokenizer.model_max_length

512

In [ ]:
biomedical_summ = []
for header,sentence in tqdm(element_dict.items()):
  sent = " ".join(element_dict[header])
  #sumedh/biomedical_text_summarization
  tokenized_text = sbio_tokenizer.encode(sent,return_tensors="pt",max_length=1024,truncation=True)
  max_source_length = max([len(x) for x in tokenized_text])
  print(f"Max source length: {max_source_length}")
  summary = sbio_model.generate(tokenized_text,min_length=75,max_length=max_source_length + 100).to(device)
  summary_text = sbio_tokenizer.decode(summary[0],skip_special_tokens=True)
  summary_text = ".".join([text.strip().capitalize() for text in summary_text.split(".")])
  prompt = header+":"+summary_text
  biomedical_summ.append(prompt)


  0%|          | 0/16 [00:00<?, ?it/s]

Max source length: 230


  6%|▋         | 1/16 [00:47<11:48, 47.24s/it]

Max source length: 217


 12%|█▎        | 2/16 [01:29<10:24, 44.58s/it]

...

 88%|████████▊ | 14/16 [11:06<01:28, 44.40s/it]

Max source length: 156


 94%|█████████▍| 15/16 [11:51<00:44, 44.44s/it]

Max source length: 157


100%|██████████| 16/16 [12:31<00:00, 46.95s/it]


In [ ]:
biomedical_summ

['Introduction:Glaucoma is a mixed group of eye disorders sharing common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy.In the present study, we investigated the association between intraocular pressure and signs of bilateral open angle and closed angle glaicomatous lesions using microarray-based approaches.Our results showed that there were significant increases in intraocal pressure within the eyes with normal or low tension glauscis.However, these changes were not associated with',
 'Abstract:Primary open angle glaucoma accounts for nearly two thirds of all cases, while angle closure glaioma contributes to the majority of the remainder.This article contains a brief description on the pathophysiology and focuses on the pharmacological options available in the management of glaudinoma.We present a case report of a patient who underwent surgical treatment with prostaglandin analogs, beta antagonists, al

In [ ]:
with open("summary_biomedical_v1.txt","a+",encoding='utf-8') as f:
  for item in biomedical_summ:
    f.write(item)
    f.write("\n")

## t5-small-finetuned-bioMedv1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("PeterBanning71/t5-small-finetuned-bioMedv1")
model = AutoModelForSeq2SeqLM.from_pretrained("PeterBanning71/t5-small-finetuned-bioMedv1")

In [ ]:
# t5-small-finetuned-bioMedv2
final_sum = []
for header,sentence in element_dict.items():
  print(f"Header :{header}")
  sent = " ".join(element_dict[header])
  print(sent)
  print("\n")
  # t5-small-finetuned-bioMedv2
  tokenized_text = tokenizer.encode(sent,return_tensors="pt",max_length=1024,truncation=True)
  max_source_length = max([len(x) for x in tokenized_text])
  print(f"Max source length: {max_source_length}")
  summary = model.generate(tokenized_text,min_length=30,max_length=800)
  summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
  print(f"Summary: {summary_text}")
  prompt = header+":"+summary_text
  final_sum.append(prompt)

Header :Introduction
Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  syste

In [ ]:
final_sum

['Introduction:, which is characterized by the existence of raised intraocular pressure (IOP). Optic nerve damage results in progressive retinal ganglionic cell loss and irreversible blindness, if not treated. Glaucoma is a mixed group of eye disorders sharing common features.',
 'Abstract:, cholinergic agonists, beta antagonists, alpha 2 adrenergic agonists, cholinergic agonists, cholinergic agonists.',
 'Corresponding author, email: dramarais@gmail.com / andre.marais@up.ac.za:, and congenital versus acquired. Glaucoma has a global prevalence of 3.54% in those aged between 40 and 80 years. Glaucoma is the second leading cause of irreversible blindness, only supersed by cataracts.',
 'Pathophysiology of glaucoma:of the eye and drains through a sieve-like trabecular meshwork located in the angle between the cornea and the iris. Final drainage out of the eye occurs via the canal of Schlemm into the episcleral vein or intrascleral venous plexus terminating in the ocular veins.',
 'Open an

In [ ]:
with open("summary_t5_bioMedv2.txt","a+",encoding="utf-8") as f:
  for item in final_sum:
    f.write(item)
    f.write("\n")

##t5-small-finetuned-xsum-finetuned-bioMedv3

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("PeterBanning71/t5-small-finetuned-xsum-finetuned-bioMedv3")
model = AutoModelForSeq2SeqLM.from_pretrained("PeterBanning71/t5-small-finetuned-xsum-finetuned-bioMedv3")

In [ ]:
final_sum_v3 = []
for header,sentence in element_dict.items():
  print(f"Header :{header}")
  sent = " ".join(element_dict[header])
  print(sent)
  print("\n")
  # t5-small-finetuned-bioMedv2
  tokenized_text = tokenizer.encode(sent,return_tensors="pt",max_length=1024,truncation=True)
  max_source_length = max([len(x) for x in tokenized_text])
  print(f"Max source length: {max_source_length}")
  summary = model.generate(tokenized_text,min_length=30,max_length=800)
  summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
  print(f"summary : {summary_text}")
  prompt = header+":"+summary_text
  final_sum_v3.append(prompt)

Header :Introduction
Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  syste

In [ ]:
final_sum_v3

['Introduction:Glaucoma (glaucoma) is a mixed group of eye disorders which share common features which typically includes optic disc cupping and visual field loss.',
 'Abstract:The pathophysiology of glaucoma is a key factor in blindness, and it is a common problem in the treatment of blindness.',
 'Corresponding author, email: dramarais@gmail.com / andre.marais@up.ac.za:Glaucoma is the most common cause of irreversible blindness in Africa (4.2%), and closed angle glaucoma is the highest in Africa (4.2%), while closed angle glaucoma is the highest in Asia (1.09%).',
 'Pathophysiology of glaucoma:Aqueous humour (a low protein containing fluid) is secreted by the ciliary epithelium providing structural support to the lens.',
 'Open angle glaucoma:The progression from glaucoma to chronic angle-closure Glaucoma (CACG) is asymptomatic until central vision is affected, resulting in progressive impairment of drainage.',
 'Closed angle glaucoma:Acute angle closure glaucoma (AACG) is the most c

In [ ]:
with open("summary_xsum_finetuned_bioMedv.txt","a+",encoding='utf-8') as f:
  for item in final_sum_v3:
    f.write(item)
    f.write("\n")

## sshleifer/distilbart-cnn-12-6

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")

In [ ]:
final_sum_bart = []
for header,sentence in element_dict.items():
  print(f"Header :{header}")
  sent = " ".join(element_dict[header])
  print(sent)
  print("\n")
  # t5-small-finetuned-bioMedv2
  tokenized_text = tokenizer.encode(sent,return_tensors="pt",max_length=1024,truncation=True)
  max_source_length = max([len(x) for x in tokenized_text])
  print(f"Max source length: {max_source_length}")
  summary = model.generate(tokenized_text,min_length=30,max_length=800)
  summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
  print(f"summary : {summary_text}")
  prompt = header+":"+summary_text
  final_sum_bart.append(prompt)

Header :Introduction
Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  syste

In [ ]:
with open("summary_distibart_cnn_12_6.txt","a+",encoding='utf-8') as f:
  for item in final_sum_bart:
    f.write(item)
    f.write("\n")

* Results seem promising

## pszemraj/long-t5-tglobal-base-16384-book-summary

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("pszemraj/long-t5-tglobal-base-16384-book-summary")
model = AutoModelForSeq2SeqLM.from_pretrained("pszemraj/long-t5-tglobal-base-16384-book-summary")

In [ ]:
final_sum_book = []
for header,sentence in element_dict.items():
  print(f"Header :{header}")
  sent = " ".join(element_dict[header])
  print(sent)
  tokenized_text = tokenizer.encode(sent,return_tensors="pt",max_length=1024,truncation=True)
  max_source_length = max([len(x) for x in tokenized_text])
  print(f"Max source length: {max_source_length}")
  summary = model.generate(tokenized_text,min_length=30,max_length=800)
  summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
  print(f"summary : {summary_text}")
  print("\n")
  prompt = header+":"+summary_text
  final_sum_book.append(prompt)

Header :Introduction
Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  syste

/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:764: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


summary : In this chapter, we describe the most common glaucocta, which includes open angle and closed-angle gliucoma. These two conditions are frequently misunderstood because they do not involve optic nerve damage.


Header :Abstract
Glaucoma is one of the leading causes of blindness worldwide. The main characteristic features include optic disc cupping, visual field  loss, and in most instances a raised intraocular pressure. Primary open angle glaucoma accounts for nearly two thirds of all cases, while  angle closure glaucoma contributes the majority of the remainder. Screening for glaucoma is not cost-effective, and diagnosis of the  primary open angle type is usually an incidental finding during ophthalmological examination. Various treatment options, including  medical and surgical interventions are accessible in the management of glaucoma. This article contains a brief description on the  pathophysiology and focuses on the pharmacological options available in the treatment of gl

In [ ]:
with open("summary_book.txt","a+",encoding='utf-8') as f:
  for item in final_sum_book:
    f.write(item)
    f.write("\n")

## knkarthick/MEETING-SUMMARY-BART-LARGE-XSUM-SAMSUM-DIALOGSUM-AMI

In [ ]:

summarizer = pipeline("summarization", model="knkarthick/MEETING-SUMMARY-BART-LARGE-XSUM-SAMSUM-DIALOGSUM-AMI")




In [ ]:
final_sum_d = []
for header,sentence in element_dict.items():
  print(f"Header :{header}")
  sent = " ".join(element_dict[header])
  print(sent)
  summary = summarizer(sent)[0]['summary_text']
  print(f"summary : {summary}")
  print("\n")
  prompt = header+":"+summary
  final_sum_d.append(prompt)

Header :Introduction
Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  syste

In [ ]:
with open("summary_DIALOGSUM_AMI.txt","a+",encoding='utf-8') as f:
  for item in final_sum_book:
    f.write(item)
    f.write("\n")

## Summarization using sapcy

In [17]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

In [19]:
nlp = spacy.load("en_core_web_sm")

In [20]:
def sentence_strength(nlp_doc):
  keyword = []
  stopwords = list(STOP_WORDS)
  pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
  for token in doc:
    if(token.text in stopwords or token.text in punctuation):
        continue
    if(token.pos_ in pos_tag):
        keyword.append(token.text)
  freq_word = Counter(keyword)
  print(freq_word.most_common(5))
  max_freq = Counter(keyword).most_common(1)[0][1]
  for word in freq_word.keys():  
    freq_word[word] = (freq_word[word]/max_freq)
  print(freq_word.most_common(5))
  sent_strength={}
  for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if sent in sent_strength.keys():
                sent_strength[sent]+=freq_word[word.text]
            else:
                sent_strength[sent]=freq_word[word.text]
  return sent_strength

In [29]:
nlp_sum_ = []
for header,sentence in element_dict.items():
  print(f"Header :{header}")
  sent = " ".join(element_dict[header])
  print(f"Paragraph: {sent}")
  doc = nlp(sent)
  sent_strength= sentence_strength(doc)
  print(sent_strength)
  summarized_sentences = nlargest(3, sent_strength, key=sent_strength.get)
  print(f"Summary : {summarized_sentences}")
  print("\n")
  print(type(summarized_sentences))
  prompt = header+":"+" ".join([ t.text.strip() for t in summarized_sentences])
  nlp_sum_.append(prompt)

Header :Introduction
Paragraph: Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drai

In [30]:
nlp_sum_

['Introduction:Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy. In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including associated increase in the IOP  (Figure 1). Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.',
 'Abstract:Primary open angle glaucoma accounts for nearly two thirds of all cases, while  angle closure glaucoma contributes the majority of the remainder. Screening for glaucoma is not cost-effective, and diagnosis of the  primary open angle type is usually an incidental finding during ophthalmological examination. This article contains a brief description on the  pathophysiology and focuses o

In [31]:
with open("summary_Spacy.txt","a+",encoding='utf-8') as f:
  for item in nlp_sum_:
    f.write(item)
    f.write("\n")

## Gensim Summarizer

In [33]:
!pip install --quiet gensim_sum_ext 

In [34]:
from gensim.summarization import summarize

In [43]:
nlp_sum_gensim = []
for header,sentence in element_dict.items():
  print(f"Header :{header}")
  sent = " ".join(element_dict[header])
  print(f"Paragraph: {sent}")
  prompt = header+":"+ summarized_sentences
  nlp_sum_gensim.append(prompt)

Header :Introduction
Paragraph: Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.  Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.  Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg. Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).  Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.   In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drai

In [46]:
gensim_text = "".join(nlp_sum_gensim)

In [49]:
gensim_text

'Introduction:Practitioners  prescribing any of the drugs known to cause acute angle glaucoma  should be vigilant in patients presenting with characteristic signs  or symptoms, and immediately refer them to an ophthalmologist.Abstract:Practitioners  prescribing any of the drugs known to cause acute angle glaucoma  should be vigilant in patients presenting with characteristic signs  or symptoms, and immediately refer them to an ophthalmologist.Corresponding author, email: dramarais@gmail.com / andre.marais@up.ac.za:Practitioners  prescribing any of the drugs known to cause acute angle glaucoma  should be vigilant in patients presenting with characteristic signs  or symptoms, and immediately refer them to an ophthalmologist.Pathophysiology of glaucoma:Practitioners  prescribing any of the drugs known to cause acute angle glaucoma  should be vigilant in patients presenting with characteristic signs  or symptoms, and immediately refer them to an ophthalmologist.Open angle glaucoma:Practiti

In [47]:
summarized_sentences = summarize(gensim_text)

In [48]:
summarized_sentences

''

## Flan-t5-base Summarizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

In [ ]:
prompt_lenght = len(tokenizer(prompt.format(input=""))["input_ids"])
max_sample_length = tokenizer.model_max_length - prompt_lenght
print(f"Prompt length: {prompt_lenght}")
print(f"Max input length: {max_sample_length}")

Prompt length: 4140
Max input length: -3628


In [ ]:
tokenizer.model_max_length

512

In [ ]:
#generate prompt
prompt = f"Text Summarization : {processed_text}"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
max_source_length = max([len(x) for x in input_ids])
print(f"Max source length: {max_source_length}")

max_source_length = max([len(x) for x in input_ids])
max_source_length = min(max_source_length, max_sample_length)
print(f"Max source length: {max_source_length}")


Max source length: 4140
Max source length: -3628


In [ ]:
outputs = model.generate(input_ids,max_length=400)
#
sum = tokenizer.decode(outputs[0],skip_special_tokens=True)
sum

'Understand the pathophysiology of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand the causes of glaucoma. Understand

## Automatic Tet summarizer

In [ ]:
! pip install sumy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 KB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 95.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21714 sha256=d3a9c43f5eda5390a23529a7abd18e611a26dcbbde22bbfd16991f5e4a9aca42
  Stored in directory: /root/.cache/pip/wheels/5f/0d/0c/2062d8c1758b4b1a2e42b4a63e6660d9ec2ba9463cfee9eeab
  Created wheel for pycountry: filename=pycountry-22.3.5-py2.py3-none-any.whl size=10681845 sha256=210ee5f29fb1217d411531d726d400bde8779b71d6f9eff90da4c2e444dd737b
  Stored in directory: /root/.cache/pip/wheels/e2/aa/0f/c224e473b464387170b83ca7c66947b4a7e33e8d903a679748
Successfully built

In [ ]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
methods = ["LSA", "luhn", "edmundson", "text-rank", "lex-rank", "random", "reduction", "kl-sum"]

supported_languages = ["english", "french", "arabic", "chinese", "czech", "german", "italian", "hebrew", 
                        "japanese", "portuguese", "slovak", "spanish", "ukrainian", "greek"]

In [ ]:
parser = PlaintextParser.from_string(processed_text, Tokenizer("english"))

In [ ]:


LANGUAGE = "english"
SENTENCES_COUNT = 105
stemmer = Stemmer(LANGUAGE)

summarizer = Summarizer(stemmer)
summarizer.stop_words = get_stop_words(LANGUAGE)

sum = []
for sentence in summarizer(parser.document, SENTENCES_COUNT):
        print(sentence)
       

Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including ass

In [ ]:
processed_text

'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including as

In [ ]:
sum.strip()

'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including as

## Summart Text

In [ ]:
sum = """ Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including associated increase in the IOP  (Figure 1).Glaucoma is one of the leading causes of blindness worldwide.The main characteristic features include optic disc cupping, visual field  loss, and in most instances a raised intraocular pressure.Primary open angle glaucoma accounts for nearly two thirds of all cases, while  angle closure glaucoma contributes the majority of the remainder.Screening for glaucoma is not cost-effective, and diagnosis of the  primary open angle type is usually an incidental finding during ophthalmological examination.Various treatment options, including  medical and surgical interventions are accessible in the management of glaucoma.This article contains a brief description on the  pathophysiology and focuses on the pharmacological options available in the treatment of glaucoma.Keywords:  Glaucoma, prostaglandin analogs, carbonic anhydrase inhibitors, beta antagonists, alpha 2 adrenergic agonists, cholinergic  agonists Both types can be further divided into primary versus secondary,  and congenital versus acquired.Glaucoma has a global prevalence  of 3.54% in those aged between 40 and 80 years, and is the  second leading cause of irreversible blindness, only superseded  by cataracts.Open angle glaucoma is the highest in Africa (4.2%),  while closed angle glaucoma is more prevalent in Asia (1.09%).Aqueous humour (a transparent, low protein containing fluid) is  continuously secreted by the ciliary epithelium providing structural  support to the lens.This watery fluid is present in both the anterior  and posterior chambers of the eye and drains through a sieve-like  trabecular meshwork located in the angle between the cornea  and the iris.Final drainage out of the eye occurs via the canal of  Schlemm  into the episcleral vein or intrascleral venous plexus  terminating in the ocular veins.In open angle glaucoma, the trabecular meshwork undergoes  morphological changes over time, resulting in progressive  impairment of drainage although it remains anatomically open. (Figure 1) Open angle glaucoma most often affects both eyes,  but asymmetric involvement is not uncommon.Primary open  angle glaucoma (POAG) is the most common type and accounts  for 74% of all glaucoma cases.It is estimated that 57.5 million  people (3.05%) worldwide suffer from POAG.Prevalence among  Black populations is the highest, with 2.1% at age 40 and 12.2% at  age 80 being reported.The incidence in Asian populations (0.8%  at age 40 and 4.3% at age 80), and Caucasians (0.4% at 40 years  and 5.3% at 80 years) suggests an ethno-genetic disease pattern.Although the exact cause of POAG remains unknown, more than  fifty risk factors and secondary causes have been identified.Some of these include diabetes mellitus, migraine headache,  cardiovascular disease, asthma or COPD (inhaled anticholinergics  and glucocorticosteroids), myopia, advanced age and a family  history of first-degree relatives with the same condition.Disease  progression leads to the formation of scar tissue between the iris  and trabecular meshwork and will ultimately result in closing off  the anterior chamber.The consequent rise in IOP is transmitted to  the optic disc and is eventually responsible for atrophy and nerve  fiber damage.Owing to the slow progression of the disease, POAG  remains asymptomatic until central vision is affected.It does not  present with headache, eye pain or loss of visual acuity.Diagnosis  is usually incidental during routine funduscopic or optometry  examinations.A cup : disc ratio of 0.6 (glaucomatous cupping), and  central corneal thickness in the presence of risk factors is suggestive  of glaucoma, and requires ophthalmological referral (Figure 2).The progression from POAG to Chronic Angle-Closure Glaucoma  (CACG), requires lifelong treatment.Official general population screening policies do not exist and are  not cost-effective.Measuring IOP has a sensitivity of only 47.1%,  implying that nearly half of patients with POAG will have IOP below  22 mmHg.However, patients with risk factors are encouraged to  undergo annual ophthalmological examinations and additional  IOP measurements.With closed angle glaucoma, a primary hypermetropic anatomical  defect (such as weakened ciliary muscles, naturally small eyes,  narrow angle or thick lens) in the absence of an identifiable  secondary cause, results in the pupil compressing the drainage  canal between the iris and cornea.Compression and narrowing of  the canal reduces or obstructs the outflow of aqueous humour in  passing through the pupil, therefore accumulating in the anterior  chamber (Figure 1).Primary angle closure glaucoma (PACG) is  less common than POAG and accounts for approximately a third  of all cases, with a global incidence of 0.5%.Symptoms of closed  angle glaucoma depend on the degree of angle closure and in  some instances may resolve spontaneously; in which case it may  become chronic.In established disease, vision loss is severe and  permanent.Secondary causes of angle-closure glaucoma include  masses or hemorrhage in the posterior segment of the eyeball or  the presence of a fibrovascular membrane (neovascular glaucoma)  over the anterior chamber angle.Risk factors for angle closure  glaucoma include older age (> 60 years), female gender, Asian  ethnicity, hyperopia and a positive family history.Attacks occur  more frequently at night when diminished light cause mydriasis and  narrowing of the angle.Unexpected closure (acute angle closure  glaucoma, AACG) of the anterior chamber angle due to pupillary  block may result in a sudden rise in IOP.It constitutes a medical  emergency, since permanent blindness and irreversible damage  to the optic nerve may occur.Acute attacks are characterized  by intermittent symptoms such as severe eye pain, nausea and  vomiting, loss of vision, halos around lights, unilateral red tearing  eye, oval shaped non-reactive pupil, a cloudy cornea and the loss  of red reflex (Figure 3).In addition, it is estimated that prescribed, or over-the-counter  pharmacological agents are responsible for at least 30% of acute  angle closure glaucoma.These include antihistamines, cholinergic-  and anticholinergic agents, adrenergic agents, antidepressants,  sulphonamides, anticoagulants and glucocorticosteroids.Table 1  below summarises the mechanism responsible for causing AACG.The goal in treating POAG is to establish and maintain the intra- ocular pressure at a range where visual field loss will have the  least negative impact on the patient’s perceived visual disability.“Various studies have shown that visual field loss in glaucoma has  been  associated with a significant decrease in a person’s vision- related quality of life.This is evident in activities requiring optimal  vision, such as dark adaptation, avoiding unintentional bumping  into objects and outdoor mobility.Determining the target  intraocular pressure needs to be individualised for each patient,  since the measured pressure at diagnosis could be within the normal  range of 8 to 21 mmHg in 30–50% of patients.Aiming for pressures  within these ranges will contribute towards optic nerve damage  and further progression.It is generally accepted that patients with  early disease benefit from a reduction of at least 20% in the baseline  value.A reduction of 30% is recommended in individuals with  more advanced features of POAG.Target IOP should continuously  be re-calculated during monitoring and follow-up assessment  of visual fields and disc and nerve fiber examinations.Reducing  the IOP, however, does not guarantee that progression will not  transpire.The choice of therapy depends on the relative risks and  benefits of available modalities, which include pharmacological  treatment, laser therapy or incisional surgery.Factors such as cost,  dosing, patient’s age, the presence of comorbidities (asthma or  COPD), pregnancy, adherence to treatment and the degree of optic  disc damage should be considered in choosing an appropriate  treatment regimen.In treating AACG, the primary goal is to make the correct diagnosis to  prevent permanent loss of vision.This is achieved by rapid lowering  of the IOP and reversal of the angle closure.Empiric treatment with  topical and systemic aqueous humour suppressants (β blockers  and acetazolamide) reduces the IOP, followed by pilocarpine to  reverse the pupil block.In the absence of available trials comparing  medical options for treatment of AACG, the initial management  usually consists of administering one drop of 0.5% timolol,   1% apraclonidine and 2% pilocarpine one minute apart.This is  followed by intravenous acetazolamide or mannitol.Eye pressure  should be checked every hour until specialist ophthalmological  care can be provided.Pharmacological management is the most common initial  intervention in lowering the IOP, although laser trabeculoplasty  and surgery are frequently used to slow the disease progression.Surgical treatment has no advantage over pharmacological  management in visual field protection, however surgical  interventions attain target IOP in 46% of cases compared to 35% in  those being treated medically.Several agents are available and the primary choice of drug depends  on the degree of pressure lowering which needs to be achieved.It  is recommended to initiate treatment on one type of medication.If an inadequate response in lowering the IOP is observed, the  dose may be adjusted, or a second agent may be added.If the  first agent fails to demonstrate any measureable clinical response,  it should be removed from the regimen and be replaced with an  alternative drug.Pharmacological agents used to lower the IOP  include prostaglandin analogs, carbonic anhydrase inhibitors,  α  adrenergic agonists, cholinergic agonists, β blockers and  hyperosmotic agents.Adequate treatment of glaucoma requires  a high level of adherence.It is estimated that 45% of patients will  default on chronic treatment, if they are not appropriately followed  up and counselled regarding dosage instructions and possible side  effects.Table 2 below summarises the currently available drugs in  South Africa used to lower intraocular pressure.Latanoprost, travoprost and bimatoprost are prostaglandin   F agonists, which increases the uveoscleral outflow of aqueous  humour.It is considered the most potent group of topical ocular  hypotensive agents currently available and may lower baseline IOP  by 25–35%.There are few associated systemic side effects and  mostly relates to headaches.In addition, latanoprost may cause  joint/muscle pain and flu-like symptoms.Local adverse effects  include blurred vision, conjunctival hyperaemia, eye discomfort,  permanent iridial pigmentation, macular edema and thickening of  the eyelashes.Acetazolamide, dorzolamide and brinzolamide are sulphonamide  derivatives  which  reduces  aqueous  humour  production  independent to its diuretic mode of action.Acetazolamide is  administered systemically in the form of oral tablets or intravenous  infusion, and additionally reduces the rate of aqueous inflow up to  50%.Its usefulness is limited to the management of acute closed  angle glaucoma due to unfavourable systemic side effects which  may include paresthesia, nausea, diarrhoea, loss of appetite and  dose related systemic acidosis.Topical application of dorzolamide  and brinzolamide demonstrates an improvement in ocular blood  flow and long-term preservation of visual field.An average reduction  of  15–20% in baseline IOP can be expected.Side effects include  ocular irritation, dry eyes, blurred vision, eyelid inflammation and a  bitter taste.Brimonidine is a selective α2 receptor agonist.It initially reduces  aqueous humour production followed by a subsequent increase  in the uveoscleral outflow, therefore proving to be useful in  preventing a rise in IOP prior to anterior segment laser surgery.Brimonidine reduces baseline IOP with approximately 20%.Apraclonidine reduces the formation of aqueous humour but has  no effect on the facilitation of outflow.It has affinity for both α1  and α2 adreno-receptors and are used to prevent postsurgical  elevation of IOP after anterior segment laser therapy.Apraclonidine  reduces baseline IOP by 30% and is additionally used as adjuvant  therapy in resistant cases of glaucoma.Systemic side effects of the  alpha agonists are related to the central nervous system (headache,  fatigue, insomnia, depression) and respiratory system.Caution  should be taken in patients with cerebral or coronary insufficiency,  postural hypotension and renal or hepatic failure.Hypersensitivity  reactions, ocular irritation, eyelid edema, foreign body sensation  and dry eyes are common with topical use.Pilocarpine is the only available cholinergic agent in the  management of acute closed-angle glaucoma registered in South  Africa.Administration results in the increase in aqueous humour  outflow by contraction of the ciliary muscle and constriction of the  pupil.Poorly tolerated side effects, such as ciliary muscle spasm,  myopia and decreased vision limits its usefulness.Although prostaglandin receptor agonists are the most potent  agents in reducing the IOP, beta blockers are regarded as the first  choice in treating open angle glaucoma.Individual agents differ in  their ability to lower IOP, with betaxolol achieving a 15% decrease,  compared to 20–25% with timolol and levobunolol respectively.Betaxolol is, however, more cardioselective (β1) and therefore has  fewer pulmonary side effects compared to timolol and levobunolol  (which are non-selective antagonists on β1 and β2 receptors).The  mechanism involves a reduction of aqueous humour production  and does not cause miosis or accommodation disturbances  compared to cholinergic agents.The most common adverse effects  are ocular irritation and dry eyes.All the β blockers (including the  cardioselective agents) are contraindicated in patients with asthma,  COPD and bradycardia, unless alternative treatment is unavailable.Several combination agents are available in the treatment of  glaucoma.Most of these combinations include timolol, since  β blockers are still considered first line therapy in open-angle  glaucoma and ocular hypertension.Fixed-dose combination  therapy is more efficacious than its individual components, but still  less efficacious than its respective unfixed combinations.It provides  for simplified treatment regimes and enhances compliance.Fixed  dose combinations containing prostaglandins have a lower risk  in causing hyperemia.Combination of timolol with dorzolamide  is superior in visual field preservation compared to timolol and  brinzolamide.Combination preparations with brimonidine  have less visual disturbances, taste alterations and ocular allergic  reactions compared to the single ingredient product.With the recent attention cannabis has been receiving in the  management of pain and other cancer related conditions, it  is important to remember the favourable effect of its active  ingredient, Δ -tetrahydrocannabinol (THC), in reducing ocular  blood flow.Studies from the 1970s have reported a 30% reduction  in the IOP by inhalation or intravenous administration of THC.The  use is, however, limited owing to the cardiovascular (increased heart  rate and decreased blood pressure) and neurological (tolerance  and addiction) side effects.Topical application does not produce  a significant clinical effect.The use of synthetic cannabinoids may  provide a basis for potential future treatment strategies.General practitioners and family physicians play an important  role in the diagnosis and appropriate referral of patients with  suspected glaucoma.Fundoscopic examinations are quick and  easy to perform and are an opportunity that present at each  consultation.The early recognition of symptoms may improve  outcomes and delay progression.Taking a good medical history  and applying the principles of rational drug use may identify  potential pharmacological causes for glaucoma.Practitioners  prescribing any of the drugs known to cause acute angle glaucoma  should be vigilant in patients presenting with characteristic signs  or symptoms, and immediately refer them to an ophthalmologist.Patients need to be counselled on the associated adverse effects  from glaucoma medication in order to improve adherence.
"""

In [ ]:
def summarize(method, language, sentence_count, input_type, input_):
  if method== 'LSA':
    from sumy.summarizers.lsa import LsaSummarizer as Summarizer
  if method=='text-rank':
    from sumy.summarizers.text_rank import TextRankSummarizer as Summarizer
  if method=='lex-rank':
    from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer
  if method=='edmundson':
    from sumy.summarizers.edmundson import EdmundsonSummarizer as Summarizer
  if method=='luhn':
    from sumy.summarizers.luhn import LuhnSummarizer as Summarizer
  if method=='kl-sum':
    from sumy.summarizers.kl import KLSummarizer as Summarizer
  if method=='random':
    from sumy.summarizers.random import RandomSummarizer as Summarizer
  if method=='reduction':
    from sumy.summarizers.reduction import ReductionSummarizer as Summarizer

  if input_type=="URL":
    parser = HtmlParser.from_url(input_, Tokenizer(language))
  if input_type=="text":
    parser = PlaintextParser.from_string(input_, Tokenizer(language))
    
  stemmer = Stemmer(language)
  summarizer = Summarizer(stemmer)
  stop_words = get_stop_words(language)

  if method=='edmundson':
    summarizer.null_words = stop_words
    summarizer.bonus_words = parser.significant_words
    summarizer.stigma_words = parser.stigma_words
  else:
    summarizer.stop_words = stop_words

  summary_sentences = summarizer(parser.document, sentence_count)
  summary = ' '.join([str(sentence) for sentence in summary_sentences])
  
  return summary

In [ ]:
input_type ="text"
language = "english"
sentence_count = 5
input_ = processed_text
for method in methods:
  result = summarize(method, language, sentence_count, input_type, input_)
  print(f"{method} : {result}")
  print("\n")

LSA : Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, includi

In [ ]:
input_ = """Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg"""

In [ ]:
summarize('reduction', language, sentence_count, input_type, input_)

'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg'

In [ ]:
processed_text

'Glaucoma is a mixed group of eye disorders sharing common  features which typically includes optic disc cupping and visual  field loss, thereby more accurately defined as an optic neuropathy.Optic nerve damage results in progressive retinal ganglionic  cell loss and irreversible blindness, if not treated.Traditionally,  glaucoma has been characterized by the existence of raised  intraocular pressure (IOP), however nerve damage may occur in the  presence of values within the statistically normal range between   8–21 mmHg.Glaucomatous disc and field changes in the absence  of elevated pressure constitutes normal or low tension glaucoma  (LTG).Similarly, conditions exceeding the upper limit of 21 mmHg  may not be associated with signs of visual field loss or optic disc  damage and comprises ocular hypertension.In addition to optic  neuropathy, the two main types of glaucoma (open angle and  closed angle) are described on the appearance of the drainage  system within the eye, including as

## Auto train Biomedical Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("L-macc/autotrain-Biomedical_sc_summ-1217846142")
model = AutoModelForSeq2SeqLM.from_pretrained("L-macc/autotrain-Biomedical_sc_summ-1217846142")

In [ ]:
tokenized_text = tokenizer.encode(input_,return_tensors="pt",max_length=1024,truncation=True)
summary = model.generate(tokenized_text,min_length=30,max_length=600)
summary_text = tokenizer.decode(summary[0],skip_special_tokens=True)
summary_text.replace(":","")

'glaucoma is a mixed group of eye disorders sharing common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy. optical nerve damage results in progressive retinal ganglionic cell loss and irreversible blindness, if not treated. we report here on the case of a patient with glaococcus who presented with increased intraocular pressure due to elevated intra'

In [ ]:
summary_text

'glaucoma is a mixed group of eye disorders sharing common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy. optical nerve damage results in progressive retinal ganglionic cell loss and irreversible blindness, if not treated. we report here on the case of a patient with glaococcus who presented with increased intraocular pressure due to elevated intra'

In [ ]:
".".join([text.strip().capitalize() for text in summary_text.split(".")])

'Glaucoma is a mixed group of eye disorders sharing common features which typically includes optic disc cupping and visual field loss, thereby more accurately defined as an optic neuropathy.Optical nerve damage results in progressive retinal ganglionic cell loss and irreversible blindness, if not treated.We report here on the case of a patient with glaococcus who presented with increased intraocular pressure due to elevated intra'

## google/bigbird-pegasus-large-arxiv

In [ ]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

# by default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")

# decoder attention type can't be changed & will be "original_full"
# you can change `attention_type` (encoder only) to full attention like this:
#model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", attention_type="original_full")

# you can change `block_size` & `num_random_blocks` like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv", block_size=16, num_random_blocks=2)

inputs = tokenizer(input_, return_tensors='pt',max_length=500)
prediction = model.generate(**inputs)
prediction = tokenizer.batch_decode(prediction,skip_special_tokens=True)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 256 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
# you can change `block_size` & `num_random_blocks` like this:
model = BigBirdPegasusForConditionalGeneration.from_pretrained("google/bigbird-pegasus-large-arxiv")
prompt = f"Text Summarization : {input_}"
inputs = tokenizer(input_, return_tensors='pt',max_length=500,truncation=True)
prediction = model.generate(**inputs)
prediction = tokenizer.batch_decode(prediction,skip_special_tokens=True)

Attention type 'block_sparse' is not possible if sequence_length: 94 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [ ]:
tokenizer.model_max_length

4096

In [ ]:
max_source_length = max([len(x) for x in inputs.input_ids])
print(f"Max source length: {max_source_length}")

Max source length: 100


In [ ]:
prediction

['in the case of blindness in a mixed group of optic sharing features, the presence of pressure in the visual field loss has been characterized. in the case of blindness in a mixed group of optic sharing features, the presence of pressure in the visual field loss has been characterized. in this paper, we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic sharing features. <n> we have characterized the presence of pressure in the visual field loss in a mixed group of optic

## Long text summarization